# IST 718: Big Data Analytics

- Professor: Daniel Acuna <deacuna@syr.edu>

## General instructions:

- You are welcome to discuss the problems with your classmates but __you are not allowed to copy any part of your answers either from your classmates or from the internet__
- You can put the homework files anywhere you want in your http://notebook.acuna.io workspace but _do not change_ the file names. The TAs and the professor use these names to grade your homework.
- Remove or comment out code that contains `raise NotImplementedError`. This is mainly to make the `assert` statement fail if nothing is submitted.
- The tests shown in some cells (i.e., `assert` and `np.testing.` statements) are used to grade your answers. **However, the professor and TAs will use __additional__ test for your answer. Think about cases where your code should run even if it passess all the tests you see.**
- Before downloading and submitting your work through Blackboard, remember to save and press `Validate` (or go to 
`Kernel`$\rightarrow$`Restart and Run All`). 
- Good luck!

In [14]:
# Load the packages needed for this part
# create spark and sparkcontext objects
from pyspark.sql import SparkSession
import numpy as np

spark = SparkSession.builder.getOrCreate()
sc = spark.sparkContext

import pyspark
from pyspark.ml import feature, regression, Pipeline, classification, pipeline, evaluation
from pyspark.sql import functions as fn, Row
from pyspark import sql

import matplotlib.pyplot as plt
import pandas as pd

# Part 2

In this section, you are going to develop a SMS spam detector based on logistic regression. This is the same idea behind sentiment analysis, but instead of predicting positive sentiment vs negative sentiment, you are going to predict whether a SMS text is spam or not.

The dataset will be in `sms_spam_df`

In [54]:
sms_spam_df = spark.read.csv('sms_spam.csv', header=True, inferSchema=True)

In [55]:
sms_spam_df.show()

+----+--------------------+
|type|                text|
+----+--------------------+
| ham|Go until jurong p...|
| ham|Ok lar... Joking ...|
|spam|Free entry in 2 a...|
| ham|U dun say so earl...|
| ham|Nah I don't think...|
|spam|FreeMsg Hey there...|
| ham|Even my brother i...|
| ham|As per your reque...|
|spam|WINNER!! As a val...|
|spam|Had your mobile 1...|
| ham|I'm gonna be home...|
|spam|SIX chances to wi...|
|spam|URGENT! You have ...|
| ham|I've been searchi...|
| ham|I HAVE A DATE ON ...|
|spam|XXXMobileMovieClu...|
| ham|Oh k...i'm watchi...|
| ham|Eh u remember how...|
| ham|Fine if that's th...|
|spam|England v Macedon...|
+----+--------------------+
only showing top 20 rows



# Question 2.1

Encode the `type` column to be 1 for `spam` and 0 for `ham` and store the result in `sms_spam2_df`

In [118]:
# create sms_spam2_df below
#sms_spam_df.select('type').distinct().show()
#encoder = OneHotEncoder(inputCols= [type], outputCols=[1,0])
sms_spam2_df = sms_spam_df.select(fn.when(fn.col("type") == 'spam', 1).otherwise(0).alias('type'),'text')
#sms_spam2_df = sms_spam_df.select(*types)
#raise NotImplementedError()

In [119]:
# check here0
sms_spam2_df.show()

+----+--------------------+
|type|                text|
+----+--------------------+
|   0|Go until jurong p...|
|   0|Ok lar... Joking ...|
|   1|Free entry in 2 a...|
|   0|U dun say so earl...|
|   0|Nah I don't think...|
|   1|FreeMsg Hey there...|
|   0|Even my brother i...|
|   0|As per your reque...|
|   1|WINNER!! As a val...|
|   1|Had your mobile 1...|
|   0|I'm gonna be home...|
|   1|SIX chances to wi...|
|   1|URGENT! You have ...|
|   0|I've been searchi...|
|   0|I HAVE A DATE ON ...|
|   1|XXXMobileMovieClu...|
|   0|Oh k...i'm watchi...|
|   0|Eh u remember how...|
|   0|Fine if that's th...|
|   1|England v Macedon...|
+----+--------------------+
only showing top 20 rows



In [120]:
# (5 pts)
np.testing.assert_array_equal(
    sms_spam2_df.groupBy('type').count().orderBy('type').rdd.map(lambda x: x['count']).collect(),
    [4827, 747]
)

# Question 2.2: tfidf feature engineering
Create a pipeline that combines a `Tokenizer`, `CounterVectorizer`, and a `IDF` estimator to compute the tfidf vectors of each SMS. Fit this pipeline and assign the pipeline transformer to a variable `tfidf_pipeline`. The `Tokenizer` step should create a column `words`, the `CounterVectorizer` step should create a column `tf`, and the `IDF` step should create a column `tfidf`.

In [220]:
# create a Pipeline transformer and name it tfidf_pipeline
# YOUR CODE HERE
tokenizer = feature.Tokenizer().setInputCol('text').setOutputCol('words')
CounterVectorizer = feature.CountVectorizer(minTF=1., minDF=5., vocabSize=2**17).setInputCol("words").setOutputCol("tf")
IDF = feature.IDF().setInputCol('tf').setOutputCol('tfidf')
tfidf_pipeline = Pipeline(stages=[tokenizer, CounterVectorizer,IDF]).fit(sms_spam2_df)
tfidf_pipeline.transform(sms_spam2_df).show(5)
type(tokenizer)
#raise NotImplementedError()

+----+--------------------+--------------------+--------------------+--------------------+
|type|                text|               words|                  tf|               tfidf|
+----+--------------------+--------------------+--------------------+--------------------+
|   0|Go until jurong p...|[go, until, juron...|(2005,[8,42,51,65...|(2005,[8,42,51,65...|
|   0|Ok lar... Joking ...|[ok, lar..., joki...|(2005,[5,74,404,5...|(2005,[5,74,404,5...|
|   1|Free entry in 2 a...|[free, entry, in,...|(2005,[0,3,8,20,5...|(2005,[0,3,8,20,5...|
|   0|U dun say so earl...|[u, dun, say, so,...|(2005,[5,22,60,14...|(2005,[5,22,60,14...|
|   0|Nah I don't think...|[nah, i, don't, t...|(2005,[0,1,66,86,...|(2005,[0,1,66,86,...|
+----+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



pyspark.ml.feature.Tokenizer

In [209]:
# (5 pts)
np.testing.assert_array_equal([type(s) for s in tfidf_pipeline.stages],
                              [feature.Tokenizer, feature.CountVectorizerModel, feature.IDFModel])

# Question 2.3: uppercase feature

Typical spam messages contain words that are upper case. Create a dataframe `sms_spam3_df` where you add a new column `has_uppercase` which contains an integer `1` if the first sequence of uppercase letters is longer or equal to 3 and an integer `0` otherwise. You can extract sequence of 3 or more uppercase letters by using the regular expression `[A-Z]{3,}`. You will use the function `fn.regexp_extract` to find those sequences and extract the first one (e.g., with index 0) and then use `fn.length` to compute the length of such sequence.

In [147]:
# create sms_spam3_df below
sms_spam3_df = sms_spam2_df.withColumn('has_uppercase',fn.when(fn.length(fn.regexp_extract(fn.col('text'),'[A-Z]{3,}',0))>=3,1).otherwise(0))#raise NotImplementedError()

In [148]:
sms_spam3_df.show()

+----+--------------------+-------------+
|type|                text|has_uppercase|
+----+--------------------+-------------+
|   0|Go until jurong p...|            0|
|   0|Ok lar... Joking ...|            0|
|   1|Free entry in 2 a...|            0|
|   0|U dun say so earl...|            0|
|   0|Nah I don't think...|            0|
|   1|FreeMsg Hey there...|            0|
|   0|Even my brother i...|            0|
|   0|As per your reque...|            0|
|   1|WINNER!! As a val...|            1|
|   1|Had your mobile 1...|            1|
|   0|I'm gonna be home...|            0|
|   1|SIX chances to wi...|            1|
|   1|URGENT! You have ...|            1|
|   0|I've been searchi...|            0|
|   0|I HAVE A DATE ON ...|            1|
|   1|XXXMobileMovieClu...|            1|
|   0|Oh k...i'm watchi...|            0|
|   0|Eh u remember how...|            0|
|   0|Fine if that's th...|            0|
|   1|England v Macedon...|            1|
+----+--------------------+-------

The first three messages with `has_uppercase == 1` are as follows:

```python
sms_spam3_df.where('has_uppercase == 1').take(3)
```

```console
[Row(type=1, text='WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.', has_uppercase=1),
 Row(type=1, text='Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030', has_uppercase=1),
 Row(type=1, text='SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info', has_uppercase=1)]
```

In [149]:
# try it here
sms_spam3_df.where('has_uppercase == 1').take(3)

[Row(type=1, text='WINNER!! As a valued network customer you have been selected to receivea £900 prize reward! To claim call 09061701461. Claim code KL341. Valid 12 hours only.', has_uppercase=1),
 Row(type=1, text='Had your mobile 11 months or more? U R entitled to Update to the latest colour mobiles with camera for Free! Call The Mobile Update Co FREE on 08002986030', has_uppercase=1),
 Row(type=1, text='SIX chances to win CASH! From 100 to 20,000 pounds txt> CSH11 and send to 87575. Cost 150p/day, 6days, 16+ TsandCs apply Reply HL 4 info', has_uppercase=1)]

In [150]:
# (5 pts)
np.testing.assert_equal(set(sms_spam3_df.columns), {'has_uppercase', 'text', 'type'})
np.testing.assert_equal(type(sms_spam3_df.schema['has_uppercase'].dataType), sql.types.IntegerType)
np.testing.assert_equal(sms_spam3_df.rdd.map(lambda x : x['has_uppercase']).sum(), 891)

# Question 2.4: Compare models

Using the following splits:

In [141]:
training_df, validation_df, testing_df = sms_spam2_df.randomSplit([0.6, 0.3, 0.1], seed=0)

In [142]:
[training_df.count(), validation_df.count(), testing_df.count()]

[3311, 1709, 554]

**(5 pts)** Create pipelines where the first stage is the `tfidf_pipeline` created above and the second stage is a `LogisticRegression` model with different regularization parameters ($\lambda$) and elastic net mixture ($\alpha$). Fit those pipelines to the appropriate data split.

1. Logistic regression with $\lambda=0$ and $\alpha=0$ (assign the fitted pipeline to `lr_pipeline1`)
2. Logistic regression with $\lambda=0.02$ and $\alpha=0.2$ (assign the fitted pipeline to `lr_pipeline2`)
3. Logistic regression with $\lambda=0.1$ and $\alpha=0.4$ (assign the fitted pipeline to `lr_pipeline3`)

In [156]:
# create lr_pipeline1, lr_pipeline2, and lr_pipeline3
lr_pipeline1 = Pipeline(stages=[tfidf_pipeline, classification.LogisticRegression().setLabelCol('type').setFeaturesCol('tfidf').setRegParam(0).setMaxIter(200).setElasticNetParam(0)]).fit(training_df)
lr_pipeline2 = Pipeline(stages=[tfidf_pipeline, classification.LogisticRegression().setLabelCol('type').setFeaturesCol('tfidf').setRegParam(0.02).setMaxIter(200).setElasticNetParam(0.2)]).fit(training_df)
lr_pipeline3 = Pipeline(stages=[tfidf_pipeline, classification.LogisticRegression().setLabelCol('type').setFeaturesCol('tfidf').setRegParam(0.1).setMaxIter(200).setElasticNetParam(0.4)]).fit(training_df)
#raise NotImplementedError()

In [155]:
# (10 pts)
np.testing.assert_equal(type(lr_pipeline1), pipeline.PipelineModel)
np.testing.assert_equal(type(lr_pipeline2), pipeline.PipelineModel)
np.testing.assert_equal(type(lr_pipeline3), pipeline.PipelineModel)
np.testing.assert_array_equal([type(s) for s in lr_pipeline1.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])
np.testing.assert_array_equal([type(s) for s in lr_pipeline2.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])
np.testing.assert_array_equal([type(s) for s in lr_pipeline3.stages],
                              [pipeline.PipelineModel, classification.LogisticRegressionModel])

AssertionError: 
Items are not equal:
 ACTUAL: <class 'pyspark.ml.pipeline.PipelineModel'>
 DESIRED: PipelineModel_757489c4a5f9

**(5 pts)** Use the evaluator object defined below to compute the area under the curve of your predictors. For example, to compute the area under the curve of pipeline 1 for a dataframe `df`, you would run

```python
evaluator.evaluate(lr_pipeline1.transform(df))
```

Assign the AUC of the three models to the variables `AUC1`, `AUC2`, and `AUC3`, and and assign the pipeline with the best model to a variable `best_model`

In [157]:
evaluator = evaluation.BinaryClassificationEvaluator(labelCol='type')

For example, the AUC on training of the first model is perfect:

```
evaluator.evaluate(lr_pipeline1.transform(training_df))
```

```console
1.0
```

In [162]:
# print the AUC for the three models as follows
# print("Model 1 AUC: ", evaluator.evaluate(....))
# etc
# finally, based on these, assign the best validated 
# model to a variable best_model
AUC1 = evaluator.evaluate(lr_pipeline1.transform(validation_df))
AUC2 = evaluator.evaluate(lr_pipeline2.transform(validation_df))
AUC3 = evaluator.evaluate(lr_pipeline3.transform(validation_df))


print("Model 1 AUC: ", AUC1)
print("Model 2 AUC: ", AUC2)
print("Model 3 AUC: ", AUC3)

best_model = evaluator.evaluate(lr_pipeline2.transform(validation_df))


#raise NotImplementedError(

Model 1 AUC:  0.9606171229900157
Model 2 AUC:  0.9900130378096487
Model 3 AUC:  0.9543473851948426


In [161]:
# (5 pts)
np.testing.assert_array_equal([type(AUC1), type(AUC2), type(AUC3)],
                             [float, float, float])
# AUC less than 1
np.testing.assert_array_less([AUC1, AUC2, AUC3], [1, 1, 1])
# AUC more than 0.5
np.testing.assert_array_less([.5, .5, .5],
                            [AUC1, AUC2, AUC3])

# Question 2.5: Choose best model

Using the right split and the best model selected before, compute the generalization performance and assign it to a variable `AUC_best`

In [165]:
# assign to AUC_best the AUC of the best model selected before
AUC_best = evaluator.evaluate(lr_pipeline2.transform(testing_df)) 
#raise NotImplementedError()

In [166]:
# (5 pts)
np.testing.assert_approx_equal(AUC_best, 
                               0.976126746201693, significant=2)

Using the same split and the best model, compute `precision`, `recall` and `f1_score`. You should first count the numbers in the confusion matrix, and then compute these metrics based on the formula.

In [285]:
# YOUR CODE HERE
sample_df = lr_pipeline2.transform(testing_df).toPandas()
sample_df.head(10)
sample_df = sample_df.drop(['text','words','tf','tfidf','rawPrediction','probability'],axis =1)
confusion_mat = pd.crosstab(sample_df['type'],sample_df['prediction'], rownames = ['Actual'] , colnames =['Predicted'],margins = True)


#raise NotImplementedError()

In [ ]:
# (5 pts)
np.testing.assert_array_almost_equal([precision, recall, f1_score],
    [1.0, 0.7976190476190477, 0.8874172185430463],
                                     decimal=2)

# Question 2.6: Inference

Use the pipeline 2 fitted above (`lr_pipeline2`) to create Pandas dataframes that contain the most negative words and the most positive words. In particular, create a dataframe `positive_words` with the columns `word` and `weight` with the top 20 positive words, sorted by descending coefficient. Similarly create a `negative_words` Pandas dataframe with the top 20 negative words where the coefficient are sorted in ascending order. **Hint: follow the `sentiment_analysis.ipynb` notebook in the repo**

In [238]:
# create positive_words and negative_words pandas dataframe below
#lr_pipeline2.transform(validation_df)
df = pd.DataFrame({'word': lr_pipeline2.stages[0].stages[1].vocabulary, 'weight': lr_pipeline2.stages[-1].coefficients.toArray()}).head(20)
negative_words = df.sort_values('weight').head(15)
positive_words = df.sort_values('weight', ascending=False).head(15)
#raise NotImplementedError()

In [239]:
# examine positive vocabulary
positive_words.head()

,word,weight
15,call,0.537161
11,your,0.187014
0,to,0.186421
10,for,0.042386
3,a,0.010809


In [240]:
# examine solutions
negative_words.head()

,word,weight
1,i,-0.183463
13,me,-0.065904
9,my,-0.016231
17,are,0.000000
16,on,0.000000


The `positive_words` and `negative_words` dataframe should look like this:

```python
positive_words.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>3555</th>
      <td>widelive.com/index.</td>
      <td>0.590870</td>
    </tr>
    <tr>
      <th>12237</th>
      <td>08714712388</td>
      <td>0.533567</td>
    </tr>
    <tr>
      <th>15</th>
      <td>call</td>
      <td>0.517100</td>
    </tr>
    <tr>
      <th>81</th>
      <td>txt</td>
      <td>0.513278</td>
    </tr>
    <tr>
      <th>9064</th>
      <td>gbp/sms</td>
      <td>0.468274</td>
    </tr>
  </tbody>
</table>

and 

```python
negative_words.head()
```

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>word</th>
      <th>weight</th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>1</th>
      <td>i</td>
      <td>-0.162493</td>
    </tr>
    <tr>
      <th>2444</th>
      <td>fighting</td>
      <td>-0.060939</td>
    </tr>
    <tr>
      <th>3221</th>
      <td>dificult</td>
      <td>-0.059061</td>
    </tr>
    <tr>
      <th>3371</th>
      <td>fightng</td>
      <td>-0.059061</td>
    </tr>
    <tr>
      <th>3332</th>
      <td>lose.</td>
      <td>-0.059061</td>
    </tr>
  </tbody>
</table>

In [241]:
# (5 pts)
np.testing.assert_equal(set(positive_words.columns), {'weight', 'word'})
np.testing.assert_equal(set(negative_words.columns), {'weight', 'word'})
np.testing.assert_approx_equal(positive_words.weight.sum(), 8.3701485692317927, significant=2)
np.testing.assert_approx_equal(negative_words.weight.sum(), -0.6661952507442954, significant=2)
np.testing.assert_array_less(positive_words.weight.iloc[-1], positive_words.weight.iloc[0])
np.testing.assert_array_less(negative_words.weight.iloc[0], negative_words.weight.iloc[-1])

AssertionError: 
Items are not equal to 2 significant digits:
 ACTUAL: 0.9637907200099549
 DESIRED: 8.370148569231793

# Question 2.7
Use the dataframe `sms_spam3_df` to create a model where the first feature is `has_uppercase` and the next set of features are the tfidf of the text. Perform feature engineering in all features using a max absolute scaler ([`MaxAbsScaler`](https://spark.apache.org/docs/2.0.2/ml-features.html#maxabsscaler)). Do a logistic regression on the resulting scaled features with regularization parameter $\lambda = 0.2$ and elastic net mixture $\alpha=0.1$ for the entire data (all of `sms_spam3_df`). Since you have scaled all features to be within the same range, you can compare them. 

**(5 pts)** with code and comments, answer below

1. is `has_uppercase` a feature that is positively or negative related to an SMS being spam?
2. what is the ratio of the coefficient of `has_uppercase` to the biggest positive tfidf coefficient?

In [296]:
spam_ID = tfidf_pipeline.transform(sms_spam2_df)

In [297]:
# your code and comments below
#sms_spam3_df = sms_spam3_df.join(spam2,spam2.text == sms_spam3_df.text)
sms_spam3_df = sms_spam3_df.select('has_uppercase','tfidf')
sms_spam3_df.show()
type(sms_spam3_df)
# YOUR CODE HERE
#raise NotImplementedError()

+-------------+--------------------+
|has_uppercase|               tfidf|
+-------------+--------------------+
|            0|(2005,[8,42,51,65...|
|            0|(2005,[5,74,404,5...|
|            0|(2005,[0,3,8,20,5...|
|            0|(2005,[0,3,8,20,5...|
|            0|(2005,[5,22,60,14...|
|            0|(2005,[0,1,66,86,...|
|            0|(2005,[0,2,6,10,1...|
|            0|(2005,[0,7,9,13,2...|
|            0|(2005,[0,10,11,44...|
|            0|(2005,[0,10,11,44...|
|            0|(2005,[0,10,11,44...|
|            1|(2005,[0,2,3,14,1...|
|            1|(2005,[0,2,3,14,1...|
|            1|(2005,[0,4,5,10,1...|
|            1|(2005,[0,4,5,10,1...|
|            0|(2005,[0,1,6,28,3...|
|            1|(2005,[0,6,40,48,...|
|            1|(2005,[0,6,40,48,...|
|            1|(2005,[0,2,3,4,8,...|
|            1|(2005,[0,2,3,4,8,...|
+-------------+--------------------+
only showing top 20 rows



pyspark.sql.dataframe.DataFrame

In [298]:
scalar = feature.MaxAbsScaler().setInputCol('features').setOutputCol('scaledFeatures')

In [299]:
model0 = Pipeline(stages=[feature.VectorAssembler(inputCols=["has_uppercase","tfidf"],
                                        outputCol='features'),scalar, classification.LogisticRegression().setLabelCol('has_uppercase').setFeaturesCol('scaledFeatures').setRegParam(0.2).setMaxIter(200).setElasticNetParam(0.1)]).fit(sms_spam3_df)

In [292]:
model0

PipelineModel_c66c0a5f87d9

In [300]:
model0.stages[2]

LogisticRegressionModel: uid=LogisticRegression_4ad8eed38a9a, numClasses=2, numFeatures=2006

In [295]:
model0.stages[2].coefficients

SparseVector(2006, {0: 2.3684, 1: 0.8634, 4: 0.1052, 8: 0.2182, 12: 0.3743, 13: 0.0751, 21: 0.089, 25: 0.2979, 30: 0.632, 32: 0.2246, 39: 0.0424, 41: 0.0919, 49: 0.1643, 50: 0.0205, 56: 0.6225, 69: 0.1358, 71: -0.0691, 82: 0.8828, 86: 0.4017, 88: 0.0169, 96: 0.6712, 97: 0.4754, 102: 0.1623, 109: 0.3348, 110: 0.4584, 126: -0.1229, 129: 0.2242, 137: 0.2413, 154: 0.3932, 168: 0.1674, 171: 0.2682, 179: 0.2635, 180: 0.3782, 204: 0.2054, 217: -0.0533, 224: 0.0559, 234: 0.1427, 235: 0.1005, 262: 0.1795, 263: 0.0139, 301: 0.2185, 302: 0.0605, 308: 0.1681, 320: 0.0482, 331: 0.1866, 353: 0.1777, 362: 0.0219, 375: 0.0922, 440: 0.0439, 464: 0.5385, 478: 0.173, 525: 0.1453, 539: 0.1326, 545: 0.1069, 555: 0.0896, 565: 0.0523, 629: 0.0537, 648: 0.0094, 653: 0.0074, 655: 0.0462, 662: 0.0462, 675: 0.0094, 775: 0.3513, 1098: 0.0085})